In [ ]:
import pandas as pd
import os
import json


BASE_DIR = '/Users/saamsani/Desktop/CMPT /soccer_score_prediction/data/football.json-master'

all_matches = []

# ------------------------- READ ALL SEASON FOLDERS -------------------------

for season_folder in sorted(os.listdir(BASE_DIR)):
    season_path = os.path.join(BASE_DIR, season_folder)

    if os.path.isdir(season_path):
        print(f" Reading season {season_folder}...")

        for filename in os.listdir(season_path):
            if filename.endswith(".json"):
                filepath = os.path.join(season_path, filename)
                with open(filepath, "r", encoding="utf-8") as f:
                    data = json.load(f)

                competition = filename.replace(".json", "")
                matches = data.get("matches", [])

                for match in matches:
                    ft_score = match.get("score", {}).get("ft")
                    if ft_score and len(ft_score) == 2:
                        home_goals = ft_score[0]
                        away_goals = ft_score[1]

                        all_matches.append({
                            "season": season_folder,
                            "competition": competition,
                            "round": match.get("round"),
                            "date": match.get("date"),
                            "team1": match.get("team1"),
                            "team2": match.get("team2"),
                            "home_goals": home_goals,
                            "away_goals": away_goals
                        })

# ------------------------- CREATE DATAFRAME -------------------------

df = pd.DataFrame(all_matches)

# Outcome column (1 = home win, 0 = draw, -1 = away win)
df["outcome"] = df.apply(
    lambda row: 1 if row["home_goals"] > row["away_goals"]
    else 0 if row["home_goals"] == row["away_goals"]
    else -1,
    axis=1
)

print("\n Sample data:")
print(df.head())

# ------------------------- SAVE TO CSV -------------------------

output_path = "/Users/saamsani/Desktop/CMPT /soccer_score_prediction/data/matches_phase1_all.csv"
df.to_csv(output_path, index=False)
print(f"\n CSV saved to {output_path} with {len(df)} matches!")
